<a href="https://colab.research.google.com/github/Storm00212/Electromagnetics/blob/main/Maxwell's_equations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is a simulation of Maxwell's equations using python code to visualize and hence better understand the concepts